In [1]:
!pip install tensorflow==1.13.1

In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/gdrive')
gitDir = "/content/gdrive/My Drive/nlp/"
os.chdir(gitDir + "data/")
print(os.listdir("."))

#sys.path.insert(0,gitDir + "data")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
['Kaggle Amazon Fine Food.ipynb', '.ipynb_checkpoints', 'Organic Dataset.ipynb', 'cache', '__pycache__', 'Amazon Reviews.ipynb', 'Amazon Multilingual.ipynb', 'tmpBACKUP.py', 'tmp.ipynb', 'amazon_multilingual.py', 'Dataset_Amazon_Multilingual.ipynb']


In [0]:
import tensorflow as tf
import os

import shutil, gzip
import pandas as pd

import numpy as np

In [0]:
def getCategory2IndexDict():
    d = {'Video DVD':                   0,
         'Music':                       1,
         'Books':                       2,
         'Mobile_Apps':                 3,
         'Digital_Video_Download':      4,
         'Digital_Music_Purchase':      5,
         'Toys':                        6,
         'Digital_Ebook_Purchase':      7,
         'PC':                          8,
         'Camera':                      9,
         'Wireless':                    10,
         'Electronics':                 11,
         'Video':                       12,
         'Sports':                      13,
         'Video Games':                 14,
         'Watches':                     15,
         'Shoes':                       16,
         'Home':                        17,
         'Musical Instruments':         18,
         'Baby':                        19,
         'Home Improvement':            20,
         'Home Entertainment':          21,
         'Office Products':             22,
         'Personal_Care_Appliances':    23,
         'Automotive':                  24,
         'Lawn and Garden':             25,
         'Luggage':                     26,
         'Kitchen':                     27,
         'Furniture':                   28,
         'Health & Personal Care':      29,
         'Software':                    30,
         'Grocery':                     31,
         'Pet Products':                32,
         'Beauty':                      33,

         #UK
         'Apparel':                     34,

         #US
         'Tools':                       35,
         'Outdoors':                    36,
         'Mobile_Electronics':          37,
         '2012-12-22':                  38,
        }
    return d

def generateFilenames(countryCode):
    x1 = "cache/amazon_multilingual_"+countryCode+"_category.npy"
    x2 = "cache/amazon_multilingual_"+countryCode+"_labels.npy"
    return (x1,x2)

def getAmazonDataFrame(filename):
    p = tf.keras.utils.get_file(filename,origin="https://s3.amazonaws.com/amazon-reviews-pds/tsv/" + filename,
                                extract=False, cache_dir=".", cache_subdir="cache")
    outDir = p[0:-3]
    if os.path.exists(outDir) == False:
        with open(outDir, 'wb') as f_out, gzip.open(p, 'rb') as f_in:
            shutil.copyfileobj(f_in, f_out)
    else:
        print(outDir, "already exists. Skipping unzipping")

    frame = pd.read_csv(p[0:-3], sep='\t', error_bad_lines=False)
    return frame


def dataToFile(data, countryCode):
    f1, f2 = generateFilenames(countryCode)
    d = getCategory2IndexDict()

    # TODO: also use headline (review_headline) ?
    x = data.review_body.values
    y = [d[x] for x in data.product_category.values]

    np.save(f1, x)
    np.save(f2, y)


In [0]:
def getData(countryCode, shuffle, buffer=None, batchsize=128):  # DE, UK, US

    f1, f2 = generateFilenames(countryCode)
    if os.path.exists(f1) and os.path.exists(f2):
        print(f1, "and", f2, " exist. Using saved npy files...")
    else:
        print(f1, "or", f2, " missing. Creating it...")
        frame = getAmazonDataFrame("amazon_reviews_multilingual_" + countryCode + "_v1_00.tsv.gz")
        dataToFile(frame, countryCode) #TODO!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    x = np.load(f1, allow_pickle=True)
    y = np.load(f2)

    print(x.shape, y.shape)
    print(x[:4])
    print(y[:4])
    print(type(y[0]))
    return buildDataset(x, y, shuffle, buffer, batchsize)



def buildDataset(x, y, shuffle, buffer, batchsize):
    length = x.shape[0]

    if buffer is None:
        buffer = length

    dataset = tf.data.Dataset.from_tensor_slices( (x,y) )
    if shuffle == True:
        # tf.data.experimental.AUTOTUNE
        dataset = dataset.shuffle(buffer).batch(batchsize).prefetch(1)
    else:
        print("dataset is not shuffled and prefetched")

    iter = dataset.make_one_shot_iterator()

    return iter, length

In [0]:
iter, length = getData("UK", shuffle=False, buffer=4, batchsize=4)

print("sample one example")

with tf.Session() as sess:
  output = sess.run(iter.get_next())
  print("out:", output)

cache/amazon_multilingual_UK_category.npy or cache/amazon_multilingual_UK_labels.npy  missing. Creating it...
./cache/amazon_reviews_multilingual_UK_v1_00.tsv already exists. Skipping unzipping


b'Skipping line 101882: expected 15 fields, saw 22\nSkipping line 115512: expected 15 fields, saw 22\n'
b'Skipping line 328404: expected 15 fields, saw 22\n'
b'Skipping line 1412642: expected 15 fields, saw 22\n'


In [8]:
!ls -alh cache

total 11G
-rw------- 1 root root 473M May 14 16:09 amazon_multilingual_DE_category.npy
-rw------- 1 root root 5.2M May 14 16:09 amazon_multilingual_DE_labels.npy
-rw------- 1 root root 2.6G May 14 16:21 amazon_multilingual_US_category.npy
-rw------- 1 root root  53M May 14 16:21 amazon_multilingual_US_labels.npy
-rw------- 1 root root 563M May 13 16:19 amazon_reviews_multilingual_DE_v1_00.tsv
-rw------- 1 root root 231M May 13 16:18 amazon_reviews_multilingual_DE_v1_00.tsv.gz
-rw------- 1 root root 821M May 14 14:03 amazon_reviews_multilingual_UK_v1_00.tsv
-rw------- 1 root root 334M May 13 16:35 amazon_reviews_multilingual_UK_v1_00.tsv.gz
-rw------- 1 root root 3.4G May 13 19:12 amazon_reviews_multilingual_US_v1_00.tsv
-rw------- 1 root root 1.4G May 13 16:27 amazon_reviews_multilingual_US_v1_00.tsv.gz
-rw------- 1 root root 9.1M May 13 10:18 reviews_Amazon_Instant_Video_5.json.gz
-rw------- 1 root root 1.6M May 13 11:17 reviews_Arts_Crafts_and_Sewing_5.json.gz
-rw------- 1 root root 

## tmp experiments

In [11]:
frame = getAmazonDataFrame("amazon_reviews_multilingual_" + "US" + "_v1_00.tsv.gz")
frame.head()

./cache/amazon_reviews_multilingual_US_v1_00.tsv already exists. Skipping unzipping


b'Skipping line 3231472: expected 15 fields, saw 22\n'
b'Skipping line 3509762: expected 15 fields, saw 22\n'
b'Skipping line 4018793: expected 15 fields, saw 22\n'
b'Skipping line 4280173: expected 15 fields, saw 22\nSkipping line 4290596: expected 15 fields, saw 22\n'
b'Skipping line 4331421: expected 15 fields, saw 22\nSkipping line 4340267: expected 15 fields, saw 22\nSkipping line 4341665: expected 15 fields, saw 22\nSkipping line 4386155: expected 15 fields, saw 22\nSkipping line 4388098: expected 15 fields, saw 22\n'
b'Skipping line 4408027: expected 15 fields, saw 22\nSkipping line 4442615: expected 15 fields, saw 22\n'
b'Skipping line 4519623: expected 15 fields, saw 22\n'
b'Skipping line 4525797: expected 15 fields, saw 22\nSkipping line 4543519: expected 15 fields, saw 22\n'
b'Skipping line 4587726: expected 15 fields, saw 22\nSkipping line 4589301: expected 15 fields, saw 22\nSkipping line 4634393: expected 15 fields, saw 22\n'
b'Skipping line 4666168: expected 15 fields, s

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,53096384,R63J84G1LOX6R,1563890119,763187671,The Sandman Vol. 1: Preludes and Nocturnes,Books,4.0,0.0,1.0,N,N,ignore the review below,this is the first 8 issues of the series. it i...,1995-08-13
1,US,53096399,R1BALOA11Z06MT,1559947608,381720534,The 22 Immutable Laws of Marketing,Books,4.0,0.0,0.0,N,N,awesome,I've always been partial to immutable laws. Th...,1995-08-17
2,US,53096332,R1LLAY5W5PZUS4,0671701800,860650224,Contact,Books,5.0,1.0,2.0,N,N,Read the book. It's good.,This is a book about first contact with aliens...,1995-08-30
3,US,53096335,R3R9VTJ82FXECQ,0425132153,624269601,Good Omens,Books,5.0,0.0,0.0,N,N,Funniest book ever written about the Anti-Christ,This is quite possibly *the* funniest book I h...,1995-09-11
4,US,51747709,R1P5J3FNBWTFXY,0517122707,161411385,A Confederacy of Dunces,Books,5.0,0.0,0.0,N,N,A winner that didn't last. Only a mothers love...,The story behind the book is almost better tha...,1995-10-17


In [17]:
frame.product_category.value_counts()

Mobile_Apps                 1467128
Digital_Ebook_Purchase      1242173
Video DVD                   1093612
Digital_Video_Download      1051622
Books                        836136
Music                        776810
Digital_Music_Purchase       107461
Toys                          57465
PC                            56817
Video                         46697
Home Entertainment            36298
Wireless                      22637
Camera                        16911
Video Games                   15398
Electronics                   11421
Musical Instruments           10914
Watches                       10537
Tools                          7475
Shoes                          7342
Baby                           5868
Sports                         4123
Home Improvement               3698
Outdoors                       3184
Office Products                2306
Home                           1998
Kitchen                        1834
Lawn and Garden                1204
Health & Personal Care      

In [0]:
def getCategory2IndexDict():
    d = {'Video DVD':                   0,
         'Music':                       1,
         'Books':                       2,
         'Mobile_Apps':                 3,
         'Digital_Video_Download':      4,
         'Digital_Music_Purchase':      5,
         'Toys':                        6,
         'Digital_Ebook_Purchase':      7,
         'PC':                          8,
         'Camera':                      9,
         'Wireless':                    10,
         'Electronics':                 11,
         'Video':                       12,
         'Sports':                      13,
         'Video Games':                 14,
         'Watches':                     15,
         'Shoes':                       16,
         'Home':                        17,
         'Musical Instruments':         18,
         'Baby':                        19,
         'Home Improvement':            20,
         'Home Entertainment':          21,
         'Office Products':             22,
         'Personal_Care_Appliances':    23,
         'Automotive':                  24,
         'Lawn and Garden':             25,
         'Luggage':                     26,
         'Kitchen':                     27,
         'Furniture':                   28,
         'Health & Personal Care':      29,
         'Software':                    30,
         'Grocery':                     31,
         'Pet Products':                32,
         'Beauty':                      33,
         
         #UK
         'Apparel':                     34,
         
         #US
         'Tools':                       35,
         'Outdoors':                    36,
         'Mobile_Electronics':          37,
         '2012-12-22':                  38,
        }
    return d
  
d=getCategory2IndexDict()

x = [d[x] for x in frame.product_category.values]